# 📋 08. openpyxl — 서식 완전 제어

> **목표**: xlwings가 못하는 **조건부 서식, 드롭다운, 이미지 삽입, 데이터 유효성 검사**를 openpyxl로 처리한다.
>
> **xlwings vs openpyxl 차이**:
> - `xlwings`: Excel을 **실시간** 제어 (Excel 실행 필요)
> - `openpyxl`: `.xlsx` 파일을 **직접 읽고 쓰기** (Excel 없어도 가능 ✅)

---

### 📚 이 챕터에서 배우는 것

| 순서 | 주제 | 실무 적용 |
|:---:|:---|:---|
| 1 | 파일/시트 기본 조작 | 생성, 열기, 저장 |
| 2 | 셀 값 & 서식 | 폰트, 배경, 테두리 |
| 3 | 조건부 서식 | 자동 색상 강조 |
| 4 | 드롭다운 & 유효성 | 입력 제한 설정 |
| 5 | 이미지 삽입 | 로고/사진 자동 삽입 |
| 6 | pandas 연동 | 분석 결과 + 정밀 서식 |
| 7 | 🔥 실전 종합 | 완성형 업무 보고서 생성 |

---
## 0️⃣ 준비: 설치 & 임포트

```bash
pip install openpyxl Pillow
```

> `Pillow`는 이미지 삽입 기능에 필요합니다.

In [ ]:
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.styles import (
    Font, PatternFill, Border, Side, Alignment, numbers
)
from openpyxl.utils import get_column_letter, column_index_from_string

print(f'✅ openpyxl 버전: {openpyxl.__version__}')
print('✅ 임포트 완료')

---
## 1️⃣ 파일 & 시트 기본 조작

> openpyxl = Excel 없이도 `.xlsx` 파일을 만들고 수정할 수 있는 라이브러리

In [ ]:
# 새 워크북 생성
wb = Workbook()
ws = wb.active       # 기본 시트
ws.title = '기본'

# 시트 추가
ws2 = wb.create_sheet('두번째시트')
ws3 = wb.create_sheet('세번째', index=0)  # 맨 앞에 삽입

print(f'📋 시트 목록: {wb.sheetnames}')

# 셀 값 입력
ws['A1'] = '안녕하세요'
ws['B1'] = 1234
ws['C1'] = 3.14
ws.cell(row=2, column=1, value='두번째 행')  # 행/열 번호로 접근

# 저장
wb.save('openpyxl_test.xlsx')
print('✅ 파일 저장 완료: openpyxl_test.xlsx')

In [ ]:
# 기존 파일 열기
wb_load = load_workbook('openpyxl_test.xlsx')
ws_load = wb_load['기본']

# 값 읽기
print(f'A1: {ws_load["A1"].value}')
print(f'B1: {ws_load["B1"].value}')
print(f'행 수: {ws_load.max_row}')
print(f'열 수: {ws_load.max_column}')

# 범위 읽기
for row in ws_load.iter_rows(min_row=1, max_row=2, values_only=True):
    print(f'  {row}')

In [ ]:
# 유용한 유틸리티 함수
print(f'열 번호 → 알파벳: {get_column_letter(3)} = C')
print(f'알파벳 → 열 번호: {column_index_from_string("D")} = 4')

# 범위 주소 만들기
start_col = get_column_letter(1)   # A
end_col   = get_column_letter(4)   # D
print(f'\n동적 범위 주소: {start_col}1:{end_col}10 = A1:D10')

---
## 2️⃣ 셀 서식 설정

> openpyxl은 서식 객체를 만들어서 셀에 할당하는 방식입니다.

In [ ]:
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment

wb2 = Workbook()
ws = wb2.active
ws.title = '서식샘플'

# ─── 폰트 ───────────────────────────────────
ws['A1'] = '굵은 빨간 글자'
ws['A1'].font = Font(
    name='맑은 고딕',
    size=14,
    bold=True,
    italic=False,
    color='FF0000'   # RRGGBB (# 없이)
)

# ─── 배경색 ──────────────────────────────────
ws['A2'] = '배경색 셀'
ws['A2'].fill = PatternFill(
    fill_type='solid',
    fgColor='FFFF00'   # 노란색
)

# ─── 테두리 ──────────────────────────────────
thin = Side(style='thin')      # 얇은 선
thick = Side(style='thick')    # 두꺼운 선

ws['A3'] = '테두리 셀'
ws['A3'].border = Border(
    top=thin, bottom=thin,
    left=thick, right=thick
)

# ─── 정렬 ────────────────────────────────────
ws['A4'] = '가운데 정렬'
ws['A4'].alignment = Alignment(horizontal='center', vertical='center')
ws.row_dimensions[4].height = 30

# ─── 숫자 서식 ────────────────────────────────
ws['A5'] = 1234567
ws['A5'].number_format = '#,##0"원"'   # 1,234,567원

ws['A6'] = 0.853
ws['A6'].number_format = '0.0%'   # 85.3%

wb2.save('style_sample.xlsx')
print('✅ 서식 샘플 저장: style_sample.xlsx')

In [ ]:
# 범위에 서식 적용 (반복문)
header_font = Font(name='맑은 고딕', size=11, bold=True, color='FFFFFF')
header_fill = PatternFill(fill_type='solid', fgColor='44729C')  # 파란색
header_align = Alignment(horizontal='center', vertical='center')
thin_side = Side(style='thin')
header_border = Border(top=thin_side, bottom=thin_side, left=thin_side, right=thin_side)

ws = wb2.active

# A10:E10 헤더 행 서식
headers = ['이름', '부서', '교육시간', '성과점수', '등급']
for col_idx, header in enumerate(headers, start=1):
    cell = ws.cell(row=10, column=col_idx, value=header)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_align
    cell.border = header_border
    ws.column_dimensions[get_column_letter(col_idx)].width = 14

ws.row_dimensions[10].height = 28
wb2.save('style_sample.xlsx')
print('✅ 헤더 서식 추가 저장 완료')

---
## 3️⃣ 조건부 서식

> Excel의 "조건부 서식"을 Python 코드로!  
> 특정 조건을 만족하는 셀에 **자동으로 색상**을 적용합니다.

> ⚠️ openpyxl 조건부 서식은 Excel의 조건부 서식 규칙을 파일에 기록하는 방식입니다.  
> Excel에서 열면 자동으로 적용됩니다!

In [ ]:
from openpyxl.formatting.rule import ColorScaleRule, DataBarRule, CellIsRule
from openpyxl.styles import PatternFill

wb3 = Workbook()
ws = wb3.active
ws.title = '조건부서식'

# 샘플 데이터
import random, pandas as pd
random.seed(42)
ws.append(['이름', '성과점수', '교육시간'])
names = ['김영희', '이철수', '박민수', '정지영', '최수진', 
         '홍길동', '강다솔', '임서연', '윤준혁', '신예린']
for name in names:
    ws.append([name, random.randint(60, 100), random.randint(5, 25)])

print('✅ 조건부 서식 데이터 입력 완료')

In [ ]:
# ─── 방법 1: 값 비교 (CellIsRule) ─────────────────
# 성과점수 90 이상 → 초록, 70 미만 → 빨강

green_fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')
red_fill   = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')
green_font = Font(color='006100')
red_font   = Font(color='9C0006')

# 90 이상이면 초록
ws.conditional_formatting.add('B2:B11',
    CellIsRule(operator='greaterThanOrEqual', formula=['90'],
               fill=green_fill, font=green_font))

# 70 미만이면 빨강
ws.conditional_formatting.add('B2:B11',
    CellIsRule(operator='lessThan', formula=['70'],
               fill=red_fill, font=red_font))

print('✅ CellIsRule 조건부 서식 추가 완료')

In [ ]:
# ─── 방법 2: 컬러 스케일 (ColorScaleRule) ─────────
# 교육시간 열: 낮으면 빨강 → 높으면 초록 자동 그라데이션

ws.conditional_formatting.add('C2:C11',
    ColorScaleRule(
        start_type='min',   start_color='FF0000',  # 최솟값 = 빨강
        mid_type='percentile', mid_value=50, mid_color='FFFF00',  # 중간 = 노랑
        end_type='max',     end_color='00FF00'    # 최댓값 = 초록
    ))

wb3.save('conditional_format.xlsx')
print('✅ ColorScaleRule 조건부 서식 추가 완료')
print('💡 conditional_format.xlsx 을 Excel에서 열어보세요!')

---
## 4️⃣ 드롭다운 (데이터 유효성 검사)

> 셀에 **선택 가능한 목록**을 제한합니다.  
> 잘못된 값 입력 방지 + 입력 편의성 향상

In [ ]:
from openpyxl.worksheet.datavalidation import DataValidation

wb4 = Workbook()
ws = wb4.active
ws.title = '드롭다운'

# 헤더
ws['A1'] = '이름'
ws['B1'] = '부서'
ws['C1'] = '등급'
ws['D1'] = '상태'

# 샘플 이름 입력
for i, name in enumerate(['김영희', '이철수', '박민수'], start=2):
    ws[f'A{i}'] = name

# ─── 드롭다운 1: 부서 선택 ───────────────────────
dv_dept = DataValidation(
    type='list',
    formula1='"영업팀,마케팅팀,개발팀,인사팀"',  # 쌍따옴표로 감싸기
    allow_blank=True,
    showDropDown=False   # False = 드롭다운 화살표 표시
)
dv_dept.prompt = '부서를 선택하세요'
dv_dept.promptTitle = '부서 선택'
ws.add_data_validation(dv_dept)
dv_dept.add('B2:B100')  # B2:B100에 적용

# ─── 드롭다운 2: 등급 선택 ───────────────────────
dv_grade = DataValidation(
    type='list',
    formula1='"A,B,C"',
    allow_blank=True,
    showDropDown=False
)
ws.add_data_validation(dv_grade)
dv_grade.add('C2:C100')

# ─── 숫자 범위 제한 (1~100) ──────────────────────
dv_num = DataValidation(
    type='whole',
    operator='between',
    formula1=1,
    formula2=100,
    showErrorMessage=True,
    errorTitle='입력 오류',
    error='1~100 사이 숫자만 입력 가능합니다'
)
ws.add_data_validation(dv_num)
dv_num.add('D2:D100')

wb4.save('dropdown_sample.xlsx')
print('✅ 드롭다운 유효성 검사 저장 완료: dropdown_sample.xlsx')
print('💡 B열 셀 클릭 시 부서 드롭다운이 나타납니다!')

---
## 5️⃣ 이미지 삽입

> 로고, 사인 이미지 등을 Excel에 자동으로 삽입합니다.

> ⚠️ `Pillow` 라이브러리가 필요합니다: `pip install Pillow`

In [ ]:
# 실습용 더미 이미지 생성 (Pillow)
try:
    from PIL import Image, ImageDraw
    
    # 100x50 로고 이미지 생성
    img = Image.new('RGB', (150, 50), color=(47, 85, 151))
    draw = ImageDraw.Draw(img)
    draw.text((10, 15), 'MY COMPANY', fill=(255, 255, 255))
    img.save('logo.png')
    print('✅ 더미 로고 이미지 생성: logo.png')
    
except ImportError:
    print('❌ Pillow 미설치 → pip install Pillow')
    print('💡 Pillow 없이도 기존 이미지 파일이 있으면 삽입 가능합니다')

In [ ]:
import os
from openpyxl.drawing.image import Image as ExcelImage

if os.path.exists('logo.png'):
    wb5 = Workbook()
    ws = wb5.active
    ws.title = '이미지삽입'
    
    ws['A1'] = '회사 보고서'
    ws['A1'].font = Font(size=16, bold=True)
    
    # 이미지 삽입
    img = ExcelImage('logo.png')
    img.width = 150    # 너비 (픽셀)
    img.height = 50    # 높이 (픽셀)
    img.anchor = 'D1'  # 삽입 위치 (셀 주소)
    
    ws.add_image(img)
    
    wb5.save('image_sample.xlsx')
    print('✅ 이미지 삽입 완료: image_sample.xlsx')
else:
    print('⏭️ logo.png 없음 → 이미지 삽입 건너뜀')

---
## 6️⃣ pandas + openpyxl 연동

> **pandas로 분석** → **openpyxl로 정밀 서식** 적용하는 최강 콤보

| 방법 | 코드 | 특징 |
|:---|:---|:---|
| `to_excel(engine='openpyxl')` | `df.to_excel('파일.xlsx')` | 빠르지만 서식 제한적 |
| `ExcelWriter` + 직접 접근 | `with pd.ExcelWriter(...) as writer:` | 서식까지 완전 제어 |

In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment

# 분석 데이터 준비
import random
random.seed(42)
departments = ['영업팀', '마케팅팀', '개발팀', '인사팀']
names_list = ['김영희', '이철수', '박민수', '정지영', '최수진',
              '홍길동', '강다솔', '임서연', '윤준혁', '신예린']

df = pd.DataFrame({
    '이름': names_list,
    '부서': [random.choice(departments) for _ in names_list],
    '교육시간': [random.randint(5, 25) for _ in names_list],
    '성과점수': [random.randint(60, 100) for _ in names_list],
    '급여': [random.randint(3000, 7000) * 1000 for _ in names_list],
})

# pandas → Excel
SAVE_PATH = 'pandas_openpyxl.xlsx'
df.to_excel(SAVE_PATH, index=False, sheet_name='원본')
print(f'✅ pandas to_excel 저장: {SAVE_PATH}')

In [ ]:
# 저장된 파일을 openpyxl로 열어 정밀 서식 추가
wb_px = load_workbook(SAVE_PATH)
ws_px = wb_px['원본']

# ─── 헤더 서식 ───────────────────────────────────
header_font = Font(name='맑은 고딕', bold=True, color='FFFFFF')
header_fill = PatternFill(fill_type='solid', fgColor='2F5597')
header_align = Alignment(horizontal='center', vertical='center')
thin = Side(style='thin')
header_border = Border(top=thin, bottom=thin, left=thin, right=thin)

for col_idx in range(1, len(df.columns) + 1):
    cell = ws_px.cell(row=1, column=col_idx)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_align
    cell.border = header_border

ws_px.row_dimensions[1].height = 30

# ─── 줄무늬 + 테두리 ──────────────────────────────
even_fill = PatternFill(fill_type='solid', fgColor='DDEEFF')
data_border = Border(top=thin, bottom=thin, left=thin, right=thin)
data_align_center = Alignment(horizontal='center', vertical='center')

for row_idx in range(2, len(df) + 2):
    row_fill = even_fill if row_idx % 2 == 0 else None
    for col_idx in range(1, len(df.columns) + 1):
        cell = ws_px.cell(row=row_idx, column=col_idx)
        if row_fill:
            cell.fill = row_fill
        cell.border = data_border
        if col_idx >= 3:  # 숫자 열은 가운데 정렬
            cell.alignment = data_align_center

# 급여 열 숫자 서식 (E열)
for row_idx in range(2, len(df) + 2):
    ws_px.cell(row=row_idx, column=5).number_format = '#,##0"원"'

# 열 너비 조정
col_widths = [12, 12, 12, 12, 16]
for col_idx, width in enumerate(col_widths, start=1):
    ws_px.column_dimensions[get_column_letter(col_idx)].width = width

wb_px.save(SAVE_PATH)
print(f'✅ 정밀 서식 적용 완료: {SAVE_PATH}')
print('💡 파일을 열어 정돈된 표 서식을 확인해보세요!')

---
## 🔥 7. 실전 종합: 완성형 업무 보고서 자동 생성

**시나리오**: 직원 데이터 → 분석 → 완성형 Excel 보고서 자동 생성  
(표지 / 원본 데이터 / 통계 요약 / 조건부 서식 포함)

```
보고서 구성
───────────
[표지] → 회사명, 보고서 제목, 날짜
[원본] → 직원 데이터 + 헤더 서식 + 줄무늬
[요약] → 부서별 통계 (groupby 결과)
[조건부서식] → 우수/미달 자동 하이라이트
```

In [ ]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
from openpyxl.formatting.rule import CellIsRule
from openpyxl.utils import get_column_letter
from datetime import date

def generate_report(df_input, output_path='monthly_report.xlsx'):
    """완성형 Excel 업무 보고서 자동 생성"""
    
    wb_r = Workbook()
    
    # 공통 스타일 정의
    thin = Side(style='thin')
    border = Border(top=thin, bottom=thin, left=thin, right=thin)
    h_fill = PatternFill(fill_type='solid', fgColor='44529C')
    h_font = Font(name='맑은 고딕', bold=True, color='FFFFFF', size=11)
    h_align = Alignment(horizontal='center', vertical='center')
    
    # ─ 시트 1: 표지 ──────────────────────────────
    ws_cover = wb_r.active
    ws_cover.title = '표지'
    
    ws_cover.column_dimensions['A'].width = 60
    ws_cover.row_dimensions[5].height = 20
    ws_cover.row_dimensions[7].height = 60
    ws_cover.row_dimensions[9].height = 40
    ws_cover.row_dimensions[11].height = 30
    
    ws_cover['A7'] = '📊 직원 교육 현황 보고서'
    ws_cover['A7'].font = Font(name='맑은 고딕', size=24, bold=True, color='2F5597')
    ws_cover['A7'].alignment = Alignment(horizontal='center', vertical='center')
    
    ws_cover['A9'] = f'보고 기준일: {date.today().strftime("%Y년 %m월 %d일")}'
    ws_cover['A9'].font = Font(name='맑은 고딕', size=14, color='595959')
    ws_cover['A9'].alignment = Alignment(horizontal='center')
    
    ws_cover['A11'] = f'대상 인원: {len(df_input)}명'
    ws_cover['A11'].font = Font(name='맑은 고딕', size=12, color='595959')
    ws_cover['A11'].alignment = Alignment(horizontal='center')
    print('✅ [1/4] 표지 시트 완료')
    
    # ─ 시트 2: 원본 데이터 ───────────────────────
    ws_data = wb_r.create_sheet('원본데이터')
    
    cols = list(df_input.columns)
    for col_idx, col_name in enumerate(cols, start=1):
        cell = ws_data.cell(row=1, column=col_idx, value=col_name)
        cell.font = h_font
        cell.fill = h_fill
        cell.alignment = h_align
        cell.border = border
        ws_data.column_dimensions[get_column_letter(col_idx)].width = 14
    ws_data.row_dimensions[1].height = 28
    
    even_fill = PatternFill(fill_type='solid', fgColor='DCE6F1')
    cal_align = Alignment(horizontal='center', vertical='center')
    
    for row_idx, (_, row) in enumerate(df_input.iterrows(), start=2):
        row_fill = even_fill if row_idx % 2 == 0 else None
        for col_idx, val in enumerate(row, start=1):
            cell = ws_data.cell(row=row_idx, column=col_idx, value=val)
            cell.border = border
            if row_fill:
                cell.fill = row_fill
            if col_idx >= 3:
                cell.alignment = cal_align
    print('✅ [2/4] 원본 데이터 시트 완료')
    
    # ─ 시트 3: 부서별 요약 ───────────────────────
    ws_sum = wb_r.create_sheet('부서요약')
    summary = df_input.groupby('부서').agg(
        인원수=('이름', 'count'),
        평균교육시간=('교육시간', 'mean'),
        평균성과점수=('성과점수', 'mean'),
        평균급여=('급여', 'mean')
    ).round(1).reset_index()
    
    sum_cols = list(summary.columns)
    for col_idx, col_name in enumerate(sum_cols, start=1):
        cell = ws_sum.cell(row=1, column=col_idx, value=col_name)
        cell.font = h_font
        cell.fill = h_fill
        cell.alignment = h_align
        cell.border = border
        ws_sum.column_dimensions[get_column_letter(col_idx)].width = 16
    ws_sum.row_dimensions[1].height = 28
    
    for row_idx, (_, row) in enumerate(summary.iterrows(), start=2):
        for col_idx, val in enumerate(row, start=1):
            cell = ws_sum.cell(row=row_idx, column=col_idx, value=val)
            cell.border = border
            cell.alignment = cal_align
    print('✅ [3/4] 부서요약 시트 완료')
    
    # ─ 시트 4: 조건부 서식 ─────────────────────
    ws_cf = wb_r.create_sheet('조건부서식')
    
    cf_cols = ['이름', '부서', '교육시간', '성과점수']
    cf_data = df_input[cf_cols].copy()
    
    for col_idx, col_name in enumerate(cf_cols, start=1):
        cell = ws_cf.cell(row=1, column=col_idx, value=col_name)
        cell.font = h_font
        cell.fill = h_fill
        cell.alignment = h_align
        cell.border = border
        ws_cf.column_dimensions[get_column_letter(col_idx)].width = 14
    ws_cf.row_dimensions[1].height = 28
    
    for row_idx, (_, row) in enumerate(cf_data.iterrows(), start=2):
        for col_idx, val in enumerate(row, start=1):
            cell = ws_cf.cell(row=row_idx, column=col_idx, value=val)
            cell.border = border
            if col_idx >= 3:
                cell.alignment = cal_align
    
    # 조건부 서식: 성과점수 (D열)
    green_fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')
    red_fill   = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')
    green_font = Font(color='006100', bold=True)
    red_font   = Font(color='9C0006', bold=True)
    
    data_range = f'D2:D{len(cf_data)+1}'
    ws_cf.conditional_formatting.add(data_range,
        CellIsRule(operator='greaterThanOrEqual', formula=['85'],
                   fill=green_fill, font=green_font))
    ws_cf.conditional_formatting.add(data_range,
        CellIsRule(operator='lessThan', formula=['70'],
                   fill=red_fill, font=red_font))
    print('✅ [4/4] 조건부 서식 시트 완료')
    
    # ─ 저장 ─────────────────────────────────────
    wb_r.save(output_path)
    print(f'\n💾 보고서 저장: {output_path}')
    print('🎉 완성형 보고서 생성 완료!')


# 실행!
df2 = pd.DataFrame({
    '이름':   names_list,
    '부서':   [random.choice(departments) for _ in names_list],
    '교육시간': [random.randint(5, 25) for _ in names_list],
    '성과점수': [random.randint(60, 100) for _ in names_list],
    '급여':   [random.randint(3000, 7000) * 1000 for _ in names_list],
})

generate_report(df2)

---
## ⚠️ 흔한 에러와 해결

| 증상 | 원인 | 해결 |
|:---|:---|:---|
| `ModuleNotFoundError: Pillow` | 이미지 삽입 시 Pillow 미설치 | `pip install Pillow` |
| `BadZipFile` 에러 | 파일이 열려있거나 손상됨 | Excel에서 파일 닫고 재시도 |
| 조건부 서식이 안 보임 | Excel에서 파일 다시 열어야 적용됨 | 파일 닫고 다시 열기 |
| 드롭다운이 사라짐 | `add_data_validation()` 누락 | `ws.add_data_validation(dv)` 확인 |
| 폰트 색상이 안 먹힘 | `#` 포함 여부 | `'FF0000'` (# 없이 6자리 HEX) |
| `load_workbook` 후 서식이 초기화됨 | `data_only=True` 옵션 | 서식 유지하려면 `data_only=False` (기본값) |

---
## 📝 연습 문제

1. `generate_report()` 를 수정해서 **표지에 오늘 날짜**를 자동으로 넣어보세요
2. 원본 데이터 시트에서 **급여 열에 `#,##0"원"` 숫자 서식**을 적용해보세요
3. 부서요약 시트에 **교육시간 평균이 15 이상인 행만 초록색**으로 조건부 서식을 추가해 보세요

In [ ]:
# 🏋️ 연습: 사용자 정의 보고서
# 원하는 데이터로 generate_report()를 호출해보세요!

# 여기에 코드 작성!


---
## 📌 여기까지 학습 완료! 🎉

### ✅ xlwings 전체 학습 로드맵 달성

```
01. 엑셀 파일 다루기 (기초)    ✅
02. 셀 다루기 (기초)           ✅
03. 셀 서식 & 스타일링         ✅
04. 반복 자동화 & 여러 시트    ✅
05. 데이터 취합 & 복사/붙여넣기 ✅
06. 실전 자동화 (차트+스케줄링) ✅
07. pandas 심화 (분석+연동)    ✅
08. openpyxl (서식 완전 제어)  ✅  ← 지금 여기!
```

### 🚀 다음 단계 추천

| 주제 | 내용 |
|:---|:---|
| **이메일 자동화** | Outlook/SMTP로 보고서 자동 발송 |
| **PyInstaller** | Python 스크립트를 `.exe`로 패키징 |
| **GUI 자동화** | `tkinter`로 보고서 생성 UI 만들기 |